## Setup

* Download the most recent version of the Bug Analysis Google Sheet
* Delete Row 1

In [31]:

import pandas as pd
import math
import numpy as np

%pip install tabulate
from tabulate import tabulate

#pip install seaborn
# %pip install seaborn

# Read the data from the TSV file
df_base = pd.read_csv('../ck_data/2024-08-17_bug_analysis_base.tsv', sep='\t')
df_rater1 = pd.read_csv('../ck_data/2024-08-17_bug_analysis_rater1.tsv', sep='\t')
df_rater2 = pd.read_csv('../ck_data/2024-08-17_bug_analysis_rater2.tsv', sep='\t')

# Drop rows were the Skipped? column is true
#df_base = df[df['Skipped?'] != True]
# Drop rows where false positives is true
#df = df[df['False pos?'] != True]

# Only keep rows where either Testing? or Set 3? is true
#can't actually delete rows bc it'll mess with the indexing
"""
df_base = df_base[(df_base['Testing?'] == True) | (df_base['Set 3?'] == True)]
df_rater1 = df_rater1[(df_rater1['Testing?'] == True) | (df_rater1['Set 3?'] == True)]
df_rater2 = df_rater2[(df_rater2['Testing?'] == True) | (df_rater2['Set 3?'] == True)]
"""

# Drop columns
#df = df.drop(columns=['Link', 'Fix Link', 'Make Benchmark?', 'Associated Benchmark', 'Rater', 'Training?', 'Testing?', 'Skipped?', 'False pos?'])
df_base = df_base.drop(columns=['Skipped?', 'Link', 'Fix Link', 'Description', 'Comments', 'Associated Benchmark'])
df_rater1 = df_rater1.drop(columns=['Skipped?', 'Link', 'Fix Link', 'Description', 'Comments', 'Associated Benchmark'])
df_rater2 = df_rater2.drop(columns=['Skipped?', 'Link', 'Fix Link', 'Description', 'Comments', 'Associated Benchmark'])

print(len(df_rater2))
df_rater2

Note: you may need to restart the kernel to use updated packages.
961



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


,Rater,Training?,Testing?,Set 3?,False pos?,Owner,Project,Title,Stars,TF-IDFs,...,Affected Computation 2,Affected Computation 3,Obscurity,Impact/Severity,# LOC,Logic Needed,BPST1,BPST2,BPST3,Make Benchmark?
0,NaN,True,False,False,False,python-poetry,tomlkit,datetime.utcnow and datetime.utcfromtimestamp ...,643.0,1.312623,...,NaN,NaN,Low,Low,Low,Low,Outdated/Deprecated APIs,NaN,NaN,Yes
1,5: Peter Sasha,False,True,False,False,frictionlessdata,frictionless-py,SpssParser ignores timezones,683.0,1.270077,...,Replacing/Rounding Datetime Components,Library Conversions,Low,Medium,Medium,Medium,Using Naïve Datetime Incorrectly,NaN,NaN,No
2,NaN,True,False,False,False,sdispater,pendulum,Deepcopy of Month-based Duration produces a di...,6109.0,1.191295,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2: Shrey Serena,False,True,False,False,googleapis,python-storage,`DeprecationWarning` warning in build log for ...,420.0,1.176351,...,NaN,NaN,Low,Low,Low,Low,Outdated/Deprecated APIs,NaN,NaN,Yes
4,NaN,True,False,False,False,googleapis,python-logging,`DeprecationWarning` warning in build log for ...,119.0,1.167279,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
956,NaN,False,False,False,False,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
957,NaN,False,False,False,False,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
958,NaN,False,False,False,False,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
959,NaN,False,False,False,False,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### False Positives

In [3]:
#calculate the number of agreed false positives (either both think it's a fp or both think not fp)
# for each pair

#rater pairs and fp counts
fp_rater_pairs = [
    ("1: Shrey Peter", 0),
    ("2: Shrey Serena", 0),
    ("3: Shrey Sasha", 0),
    ("4: Peter Serena", 0),
    ("5: Peter Sasha", 0),
    ("6: Serena Sasha", 0)
]

#convert to boolean or it wont work
df_base["Testing?"] = df_base["Testing?"].astype(bool)
df_base["Set 3?"] = df_base["Set 3?"].astype(bool)

for i in range(len(df_base)):
    if (df_base["Testing?"].iloc[i]) | (df_base["Set 3?"].iloc[i]):
        if df_rater1.iloc[i]['False pos?'] == df_rater2.iloc[i]['False pos?']:
            for j in range(len(fp_rater_pairs)):
                pair, count = fp_rater_pairs[j]
                if pair == df_base.iloc[i]['Rater']:
                    fp_rater_pairs[j] = (pair, count + 1)

#printing
for pair, count in fp_rater_pairs:
    print(pair, count)


1: Shrey Peter 24
2: Shrey Serena 20
3: Shrey Sasha 24
4: Peter Serena 27
5: Peter Sasha 25
6: Serena Sasha 26


In [41]:
#entire process: get fp agreement rate by going through each row, 
# getting the false pos values for Rater 1 and Rater 2, 
# comparing them and setting as 1 if equal and 0 otherwise,
# sum the results, and divide by number of bugs per pair

#GLOBAL VAR
NUMBER_BUGS_PER_PAIR = 30

fp_agreement = [
    ("1: Shrey Peter", 0),
    ("2: Shrey Serena", 0),
    ("3: Shrey Sasha", 0),
    ("4: Peter Serena", 0),
    ("5: Peter Sasha", 0),
    ("6: Serena Sasha", 0)
]

counter = 0
for pair, count in fp_rater_pairs:
    print(pair, count, "-->", count/NUMBER_BUGS_PER_PAIR)
    fp_agreement[counter] = (pair, count/NUMBER_BUGS_PER_PAIR)
    counter +=1

1: Shrey Peter 24 --> 0.8
2: Shrey Serena 20 --> 0.6666666666666666
3: Shrey Sasha 24 --> 0.8
4: Peter Serena 27 --> 0.9
5: Peter Sasha 25 --> 0.8333333333333334
6: Serena Sasha 26 --> 0.8666666666666667


### Categories

In [19]:
#if both raters classified the item as not fp, 
# check if the categories assigned by the 2 raters are the same

not_fp_rater_pairs = [
    ("1: Shrey Peter", 0),
    ("2: Shrey Serena", 0),
    ("3: Shrey Sasha", 0),
    ("4: Peter Serena", 0),
    ("5: Peter Sasha", 0),
    ("6: Serena Sasha", 0)
]

category_rater_pairs = [
    ("1: Shrey Peter", 0),
    ("2: Shrey Serena", 0),
    ("3: Shrey Sasha", 0),
    ("4: Peter Serena", 0),
    ("5: Peter Sasha", 0),
    ("6: Serena Sasha", 0)
]

for i in range(len(df_base)):
    if (df_base["Testing?"].iloc[i]) | (df_base["Set 3?"].iloc[i]):
        if (df_rater1.iloc[i]['False pos?']==False) & (df_rater2.iloc[i]['False pos?']==False):
            for j in range(len(not_fp_rater_pairs)):
                    pair, count = not_fp_rater_pairs[j]
                    if pair == df_base.iloc[i]['Rater']:
                        not_fp_rater_pairs[j] = (pair, count + 1)
            if(df_rater1.iloc[i]['Category 1']==df_rater2.iloc[i]['Category 1']):
                for j in range(len(category_rater_pairs)):
                    pair, count = category_rater_pairs[j]
                    if pair == df_base.iloc[i]['Rater']:
                        category_rater_pairs[j] = (pair, count + 1)

for pair, count in not_fp_rater_pairs:
    print(pair, count)

category_po = [
    ("1: Shrey Peter", 0),
    ("2: Shrey Serena", 0),
    ("3: Shrey Sasha", 0),
    ("4: Peter Serena", 0),
    ("5: Peter Sasha", 0),
    ("6: Serena Sasha", 0)
]
counter = 0
for pair, count in category_rater_pairs:
    category_po[counter] = (
        category_po[counter][0], 
        count/not_fp_rater_pairs[counter][1]
    )
    #print(pair, count, "-->", count/not_fp_rater_pairs[counter][1])
    print(pair, count, "-->", category_po[counter][1])
    counter +=1

1: Shrey Peter 18
2: Shrey Serena 19
3: Shrey Sasha 15
4: Peter Serena 19
5: Peter Sasha 18
6: Serena Sasha 19
1: Shrey Peter 15 --> 0.8333333333333334
2: Shrey Serena 4 --> 0.21052631578947367
3: Shrey Sasha 8 --> 0.5333333333333333
4: Peter Serena 16 --> 0.8421052631578947
5: Peter Sasha 12 --> 0.6666666666666666
6: Serena Sasha 17 --> 0.8947368421052632


In [20]:
# calculate the P_e score for categories
# 1) take a rater pair (A), and only the values that are !FP
# 2) for each value (category rn), count how many times it appears for rater 1 (A1) and rater 2 (A2)
# 3) values (C) are the total number of values (categories)
# 4) calculate (A1*A2 / (C*C)) for each value, and \sum them together

category_pe = [
    ("1: Shrey Peter", 0),
    ("2: Shrey Serena", 0),
    ("3: Shrey Sasha", 0),
    ("4: Peter Serena", 0),
    ("5: Peter Sasha", 0),
    ("6: Serena Sasha", 0)
]

#GLOBAL VAR
CATEGORIES = [
    "Date",
    "DST",
    "Duration",
    "String Representation",
    "Timestamps",
    "Timezone",
    "Deprecated",
    "Fold",
    "TODO"
]

categories_rater1 = [
    0, 0, 0, 0, 0, 0, 0, 0, 0 #9 values
] # for incrementing as we find values of diff categories
categories_rater2 = [
    0, 0, 0, 0, 0, 0, 0, 0, 0 #9 values
] # for incrementing as we find values of diff categories

for r in range(len(category_pe)): #for each rater pair
    for i in range(len(df_base)): #for each row in the sheet
        if(df_base["Rater"].iloc[i]==category_pe[r][0]): #if the row's Rater == the current rater pair
            if (df_rater1.iloc[i]['False pos?']==False) & (df_rater2.iloc[i]['False pos?']==False): #if it's not a fp
                for j in range(len(CATEGORIES)): #for each category
                    if(df_rater1.iloc[i]["Category 1"]==CATEGORIES[j]): #if rater 1 marked it as this category
                        categories_rater1[j] +=1 #increment count
                    if(df_rater2.iloc[i]["Category 2"]==CATEGORIES[j]): #if rater 2 marked it as this category
                        categories_rater2[j] +=1 #increment count
    tmp = 0
    for v in range(len(CATEGORIES)): #increment proportions
        tmp += ((categories_rater1[v]*categories_rater2[v]) / (len(CATEGORIES)*len(CATEGORIES))) #CHECK THIS FORMULA
        #tmp += (categories_rater1[v]/len(CATEGORIES)) * (categories_rater2[v]/len(CATEGORIES))
    category_pe[r]= (category_pe[r][0], tmp)

    #reset values
    categories_rater1 = [0, 0, 0, 0, 0, 0, 0, 0, 0]
    categories_rater2 = [0, 0, 0, 0, 0, 0, 0, 0, 0]

for pair, count in category_pe:
    print(pair, count)


1: Shrey Peter 0.5925925925925926
2: Shrey Serena 0.037037037037037035
3: Shrey Sasha 0.1111111111111111
4: Peter Serena 0.6172839506172839
5: Peter Sasha 0.037037037037037035
6: Serena Sasha 0.345679012345679


In [21]:
# calculate the cohen kappa score for categories
category_ck = [
    ("1: Shrey Peter", 0),
    ("2: Shrey Serena", 0),
    ("3: Shrey Sasha", 0),
    ("4: Peter Serena", 0),
    ("5: Peter Sasha", 0),
    ("6: Serena Sasha", 0)
]

for i in range(len(category_ck)):
    category_ck[i] = (
        category_ck[i][0],
        (category_po[i][1] - category_pe[i][1]) / (1 - category_pe[i][1])
    )

for pair, count in category_ck:
    print(pair, count)

1: Shrey Peter 0.590909090909091
2: Shrey Serena 0.18016194331983804
3: Shrey Sasha 0.47500000000000003
4: Peter Serena 0.5874363327674023
5: Peter Sasha 0.6538461538461537
6: Serena Sasha 0.8391261171797418


### Affected Computation

In [22]:
#if both raters classified the item as not fp, 
# check if the affected computation assigned by the 2 raters are the same

affcomp_rater_pairs = [
    ("1: Shrey Peter", 0),
    ("2: Shrey Serena", 0),
    ("3: Shrey Sasha", 0),
    ("4: Peter Serena", 0),
    ("5: Peter Sasha", 0),
    ("6: Serena Sasha", 0)
]

for i in range(len(df_base)):
    if (df_base["Testing?"].iloc[i]) | (df_base["Set 3?"].iloc[i]):
        if (df_rater1.iloc[i]['False pos?']==False) & (df_rater2.iloc[i]['False pos?']==False):
            if(df_rater1.iloc[i]['Affected Computation 1']==df_rater2.iloc[i]['Affected Computation 1']):
                for j in range(len(affcomp_rater_pairs)):
                    pair, count = affcomp_rater_pairs[j]
                    if pair == df_base.iloc[i]['Rater']:
                        affcomp_rater_pairs[j] = (pair, count + 1)

"""
counter = 0
for pair, count in affcomp_rater_pairs:
    print(pair, count, "-->", count/not_fp_rater_pairs[counter][1])
    counter +=1
"""

affcomp_po = [
    ("1: Shrey Peter", 0),
    ("2: Shrey Serena", 0),
    ("3: Shrey Sasha", 0),
    ("4: Peter Serena", 0),
    ("5: Peter Sasha", 0),
    ("6: Serena Sasha", 0)
]
counter = 0
for pair, count in affcomp_rater_pairs:
    affcomp_po[counter] = (
        affcomp_po[counter][0], 
        count/not_fp_rater_pairs[counter][1]
    )
    #print(pair, count, "-->", count/not_fp_rater_pairs[counter][1])
    print(pair, count, "-->", affcomp_po[counter][1])
    counter +=1

1: Shrey Peter 12 --> 0.6666666666666666
2: Shrey Serena 4 --> 0.21052631578947367
3: Shrey Sasha 4 --> 0.26666666666666666
4: Peter Serena 13 --> 0.6842105263157895
5: Peter Sasha 11 --> 0.6111111111111112
6: Serena Sasha 13 --> 0.6842105263157895


In [23]:
# calculate the P_e score for affected computations
# 1) take a rater pair (A), and only the values that are !FP
# 2) for each value (affected computations rn), count how many times it appears for rater 1 (A1) and rater 2 (A2)
# 3) values (C) are the total number of values (affcomp)
# 4) calculate (A1*A2 / (C*C)) for each value, and \sum them together

affcomp_pe = [
    ("1: Shrey Peter", 0),
    ("2: Shrey Serena", 0),
    ("3: Shrey Sasha", 0),
    ("4: Peter Serena", 0),
    ("5: Peter Sasha", 0),
    ("6: Serena Sasha", 0)
]

#GLOBAL VAR
AFFECTED_COMPUTATIONS = [
    "Timestamp/Hash Construction",
    "Datetime Arithmetic",
    "Datetime Comparison",
    "Datetime Construction",
    "Datetime Equality",
    "Delta Arithmetic",
    "Delta Comparison",
    "Delta Construction",
    "Querying Datetime Components",
    "Replacing/Rounding Datetime Components",
    "String Parsing/Formatting (humanized)",
    "String Parsing/Formatting (data)",
    "Timezone Conversions",
    "Timezone Equality",
    "Library Conversions",
    "TODO"
]

affcomp_rater1 = [0]*16 # for incrementing as we find values of diff affcomps
affcomp_rater2 = [0]*16 # for incrementing as we find values of diff affcomps

for r in range(len(affcomp_pe)): #for each rater pair
    for i in range(len(df_base)): #for each row in the sheet
        if(df_base["Rater"].iloc[i]==affcomp_pe[r][0]): #if the row's Rater == the current rater pair
            if (df_rater1.iloc[i]['False pos?']==False) & (df_rater2.iloc[i]['False pos?']==False): #if it's not a fp
                for j in range(len(CATEGORIES)): #for each affcomp
                    if(df_rater1.iloc[i]["Affected Computation 1"]==AFFECTED_COMPUTATIONS[j]): #if rater 1 marked it as this affcomp
                        affcomp_rater1[j] +=1 #increment count
                    if(df_rater2.iloc[i]["Affected Computation 2"]==AFFECTED_COMPUTATIONS[j]): #if rater 2 marked it as this affcomp
                        affcomp_rater2[j] +=1 #increment count
    tmp = 0
    for v in range(len(AFFECTED_COMPUTATIONS)): #increment proportions
        tmp += ((affcomp_rater1[v]*affcomp_rater2[v]) / (len(AFFECTED_COMPUTATIONS)*len(AFFECTED_COMPUTATIONS))) #CHECK THIS FORMULA
    affcomp_pe[r]= (affcomp_pe[r][0], tmp)

    #reset values
    affcomp_rater1 = [0]*16
    affcomp_rater2 = [0]*16

for pair, count in affcomp_pe:
    print(pair, count)


1: Shrey Peter 0.1953125
2: Shrey Serena 0.015625
3: Shrey Sasha 0.0
4: Peter Serena 0.0703125
5: Peter Sasha 0.00390625
6: Serena Sasha 0.046875


In [24]:
# calculate the cohen kappa score for affected computations
affcomp_ck = [
    ("1: Shrey Peter", 0),
    ("2: Shrey Serena", 0),
    ("3: Shrey Sasha", 0),
    ("4: Peter Serena", 0),
    ("5: Peter Sasha", 0),
    ("6: Serena Sasha", 0)
]

for i in range(len(affcomp_ck)):
    affcomp_ck[i] = (
        affcomp_ck[i][0],
        (affcomp_po[i][1] - affcomp_pe[i][1]) / (1 - affcomp_pe[i][1])
    )

for pair, count in affcomp_ck:
    print(pair, count)

1: Shrey Peter 0.5857605177993527
2: Shrey Serena 0.19799498746867167
3: Shrey Sasha 0.26666666666666666
4: Peter Serena 0.6603272888102609
5: Peter Sasha 0.6095860566448802
6: Serena Sasha 0.6686798964624676


### BPSTs

In [26]:
#if both raters classified the item as not fp, 
# check if the affected computation assigned by the 2 raters are the same

bpst_rater_pairs = [
    ("1: Shrey Peter", 0),
    ("2: Shrey Serena", 0),
    ("3: Shrey Sasha", 0),
    ("4: Peter Serena", 0),
    ("5: Peter Sasha", 0),
    ("6: Serena Sasha", 0)
]

for i in range(len(df_base)):
    if (df_base["Testing?"].iloc[i]) | (df_base["Set 3?"].iloc[i]):
        if (df_rater1.iloc[i]['False pos?']==False) & (df_rater2.iloc[i]['False pos?']==False):
            if(df_rater1.iloc[i]['BPST1']==df_rater2.iloc[i]['BPST1']):
                for j in range(len(bpst_rater_pairs)):
                    pair, count = bpst_rater_pairs[j]
                    if pair == df_base.iloc[i]['Rater']:
                        bpst_rater_pairs[j] = (pair, count + 1)

"""
counter = 0
for pair, count in bpst_rater_pairs:
    print(pair, count, "-->", count/not_fp_rater_pairs[counter][1])
    counter +=1
"""

bpst_po = [
    ("1: Shrey Peter", 0),
    ("2: Shrey Serena", 0),
    ("3: Shrey Sasha", 0),
    ("4: Peter Serena", 0),
    ("5: Peter Sasha", 0),
    ("6: Serena Sasha", 0)
]
counter = 0
for pair, count in bpst_rater_pairs:
    bpst_po[counter] = (
        bpst_po[counter][0], 
        count/not_fp_rater_pairs[counter][1]
    )
    #print(pair, count, "-->", count/not_fp_rater_pairs[counter][1])
    print(pair, count, "-->", bpst_po[counter][1])
    counter +=1

1: Shrey Peter 9 --> 0.5
2: Shrey Serena 2 --> 0.10526315789473684
3: Shrey Sasha 5 --> 0.3333333333333333
4: Peter Serena 13 --> 0.6842105263157895
5: Peter Sasha 10 --> 0.5555555555555556
6: Serena Sasha 11 --> 0.5789473684210527


In [12]:
# calculate the P_e score for bpsts
# 1) take a rater pair (A), and only the values that are !FP
# 2) for each value (bpsts rn), count how many times it appears for rater 1 (A1) and rater 2 (A2)
# 3) values (C) are the total number of values (bpst)
# 4) calculate (A1*A2 / (C*C)) for each value, and \sum them together

bpst_pe = [
    ("1: Shrey Peter", 0),
    ("2: Shrey Serena", 0),
    ("3: Shrey Sasha", 0),
    ("4: Peter Serena", 0),
    ("5: Peter Sasha", 0),
    ("6: Serena Sasha", 0)
]

#GLOBAL VAR
BPSTs = [
    "Incorrect API usage",
    "Outdated/Deprecated APIs",
    "Typo",
    "Using Naïve Datetime Incorrectly",
    "Dropping Timezones",
    "Adding Support for New Libraries",
    "Adding Support for New Features",
    "Precision of Representation",
    "Library Bug",
    "OOB Timestamps",
    "OOB Datetimes",
    "Creating time in unspecified fold",
    "Comparing Timezones",
    "Unintuitive Arithmetic",
    "Compatibility Issues",
    "Incorrect Format Assumptions",
    "TODO"
]

bpst_rater1 = [0]*17 # for incrementing as we find values of diff bpst
bpst_rater2 = [0]*17 # for incrementing as we find values of diff bpst

for r in range(len(bpst_pe)): #for each rater pair
    for i in range(len(df_base)): #for each row in the sheet
        if(df_base["Rater"].iloc[i]==bpst_pe[r][0]): #if the row's Rater == the current rater pair
            if (df_rater1.iloc[i]['False pos?']==False) & (df_rater2.iloc[i]['False pos?']==False): #if it's not a fp
                for j in range(len(BPSTs)): #for each affcomp
                    if(df_rater1.iloc[i]["BPST1"]==BPSTs[j]): #if rater 1 marked it as this bpst
                        bpst_rater1[j] +=1 #increment count
                    if(df_rater2.iloc[i]["BPST2"]==BPSTs[j]): #if rater 2 marked it as this bpst
                        bpst_rater2[j] +=1 #increment count
    tmp = 0
    for v in range(len(BPSTs)): #increment proportions
        tmp += ((bpst_rater1[v]*bpst_rater2[v]) / (len(BPSTs)*len(BPSTs))) #CHECK THIS FORMULA
    bpst_pe[r]= (bpst_pe[r][0], tmp)

    #reset values
    bpst_rater1 = [0]*17
    bpst_rater2 = [0]*17

for pair, count in bpst_pe:
    print(pair, count)


1: Shrey Peter 0.024221453287197235
2: Shrey Serena 0.01384083044982699
3: Shrey Sasha 0.0034602076124567475
4: Peter Serena 0.04498269896193772
5: Peter Sasha 0.01384083044982699
6: Serena Sasha 0.020761245674740487


In [27]:
# calculate the cohen kappa score for bpsts
bpst_ck = [
    ("1: Shrey Peter", 0),
    ("2: Shrey Serena", 0),
    ("3: Shrey Sasha", 0),
    ("4: Peter Serena", 0),
    ("5: Peter Sasha", 0),
    ("6: Serena Sasha", 0)
]

for i in range(len(bpst_ck)):
    bpst_ck[i] = (
        bpst_ck[i][0],
        (bpst_po[i][1] - bpst_pe[i][1]) / (1 - bpst_pe[i][1])
    )

for pair, count in bpst_ck:
    print(pair, count)

1: Shrey Peter 0.4875886524822695
2: Shrey Serena 0.09270544783010155
3: Shrey Sasha 0.33101851851851855
4: Peter Serena 0.6693363844393593
5: Peter Sasha 0.549317738791423
6: Serena Sasha 0.5700204575041845


### Results

In [42]:
head = ["Rater", "# Both True Positive", "False Positive %", "Category 1", "Category CK", "AffComp 1", "AffComp CK", "BPST 1", "BPST CK"]
mydata = [
    [0]*len(head),
    [0]*len(head),
    [0]*len(head),
    [0]*len(head),
    [0]*len(head),
    [0]*len(head)
]

for i in range(len(mydata)):
    mydata[i][0] = fp_rater_pairs[i][0] #Rater
    mydata[i][1] = fp_rater_pairs[i][1] #number of both true positive
    mydata[i][2] = fp_agreement[i][1] # false positive %
    mydata[i][3] = category_po[i][1] #Category 1 agreement %
    mydata[i][4] = category_ck[i][1] #Category CK
    mydata[i][5] = affcomp_po[i][1] #Affected Computation 1 agreement %
    mydata[i][6] = affcomp_ck[i][1] #Affected Computation CK
    mydata[i][7] = bpst_po[i][1] #BPST agreement %
    mydata[i][8] = bpst_ck[i][1] #BPST CK

print(tabulate(mydata, headers=head, tablefmt="grid"))

+-----------------+------------------------+--------------------+--------------+---------------+-------------+--------------+----------+-----------+
| Rater           |   # Both True Positive |   False Positive % |   Category 1 |   Category CK |   AffComp 1 |   AffComp CK |   BPST 1 |   BPST CK |
+=================+========================+====================+==============+===============+=============+==============+==========+===========+
| 1: Shrey Peter  |                     24 |           0.8      |           15 |      0.590909 |          12 |     0.585761 |        9 | 0.487589  |
+-----------------+------------------------+--------------------+--------------+---------------+-------------+--------------+----------+-----------+
| 2: Shrey Serena |                     20 |           0.666667 |            4 |      0.180162 |           4 |     0.197995 |        2 | 0.0927054 |
+-----------------+------------------------+--------------------+--------------+---------------+----------